<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Insights/blob/main/20-LangChain-ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install openai
!pip install gradio
!pip install langchain langchain_community
!pip install python-dotenv

In [ ]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [ ]:
import os
import openai
import sys

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
import gradio as gr
from datetime import datetime

# Initialize the chat model
llm = ChatOpenAI(top_p=0.1, temperature=1.0, model='gpt-3.5-turbo-0613')

# Function to save chat history to a JSON file
def save_history_to_json(history, filename="chat_history.json"):
    # Create a list to hold the formatted history
    formatted_history = []

    for human, ai in history:
        formatted_history.append({"role": "human", "content": human})
        formatted_history.append({"role": "ai", "content": ai})

    # Save the history to a JSON file
    with open(filename, "w") as f:
        json.dump(formatted_history, f, indent=4)

# Predict function with history saving
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    # Generate GPT response
    gpt_response = llm(history_langchain_format)

    # Append the new interaction to the history
    history.append((message, gpt_response.content))

    # Save the updated history to a JSON file
    save_history_to_json(history)

    return gpt_response.content

# Launch Gradio chat interface
gr.ChatInterface(predict).launch()
